In [1]:
import warnings
warnings.filterwarnings("ignore")

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

from pymongo import MongoClient
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, StructType, StructField, FloatType
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
#nltk.download('vader_lexicon')

In [2]:
# Create a SparkSession
#spark = SparkSession.builder.appName("TextProcessing").getOrCreate()
spark = SparkSession.builder \
    .appName("TextProcessing") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:2.4.2") \
    .getOrCreate()
# A JSON dataset is pointed to by path.
# Move the 'people.json' file to HDFS path '/user1'
spark.sparkContext.setLogLevel("ERROR")
path = "/user1/Trump_Tweets.json"
twitterDF = spark.read.json(path)

2023-05-21 17:34:53,859 WARN util.Utils: Your hostname, muhammad-Vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2023-05-21 17:34:53,864 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark-3.1.3-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-604c68fa-7df8-4a34-b4e9-009abd161666;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;2.4.2 in central
	found org.mongodb#mongo-java-driver;3.12.5 in central
:: resolution report :: resolve 930ms :: artifacts dl 30ms
	:: modules in use:
	org.mongodb#mongo-java-driver;3.12.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;2.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0 

In [3]:
twitterDF.printSchema()

root
 |-- Client: string (nullable = true)
 |-- Client Simplified: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [4]:
twitterDF.show()

+-------------------+-------------------+----------+-----------+--------------------+
|             Client|  Client Simplified|      Date|       Time|               Tweet|
+-------------------+-------------------+----------+-----------+--------------------+
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 6:46:22 PM| I promise that o...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 5:00:47 PM| Congratulations ...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017| 3:20:15 PM| Statement on Int...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017|11:30:29 AM| .@VP Mike Pence ...|
| Twitter for iPhone| Twitter for iPhone|01/27/2017|11:27:02 AM| The #MarchForLif...|
|Twitter for Android|Twitter for Android|01/27/2017| 8:19:10 AM| Mexico has taken...|
|Twitter for Android|Twitter for Android|01/27/2017| 8:12:52 AM| Look forward to ...|
| Twitter for iPhone| Twitter for iPhone|01/26/2017| 6:53:37 PM| Miami-Dade Mayor...|
| Twitter for iPhone| Twitter for iPhone|01/26/2017| 6

In [5]:
# Select specific columns
selected_columns = ["Date", "Time", "Tweet"]
twitterDF = twitterDF.select(*selected_columns)

In [6]:
twitterDF.show()

+----------+-----------+--------------------+
|      Date|       Time|               Tweet|
+----------+-----------+--------------------+
|01/27/2017| 6:46:22 PM| I promise that o...|
|01/27/2017| 5:00:47 PM| Congratulations ...|
|01/27/2017| 3:20:15 PM| Statement on Int...|
|01/27/2017|11:30:29 AM| .@VP Mike Pence ...|
|01/27/2017|11:27:02 AM| The #MarchForLif...|
|01/27/2017| 8:19:10 AM| Mexico has taken...|
|01/27/2017| 8:12:52 AM| Look forward to ...|
|01/26/2017| 6:53:37 PM| Miami-Dade Mayor...|
|01/26/2017| 6:45:28 PM| Will be intervie...|
|01/26/2017| 2:21:17 PM| Spoke at the Con...|
|01/26/2017| 8:55:03 AM| of jobs and comp...|
|01/26/2017| 8:51:46 AM| The U.S. has a 6...|
|01/26/2017| 6:04:24 AM| Ungrateful TRAIT...|
|01/25/2017| 9:48:25 PM| Interview with D...|
|01/25/2017| 9:45:32 PM| "@romoabcnews: ....|
|01/25/2017| 9:14:56 PM| As your Presiden...|
|01/25/2017| 7:03:33 PM| Beginning today,...|
|01/25/2017| 5:05:59 PM| I will be interv...|
|01/25/2017| 7:17:01 AM| I will be

In [7]:
# Define a function for text processing
def process_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regular expressions
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    # Remove mentions using regular expressions
    text = re.sub(r"@\w+", "", text)
    
    # Remove hashtags using regular expressions
    text = re.sub(r"#\w+", "", text)
    
    # Remove special characters and punctuation
    text = re.sub(r"[^\w\s]", "", text)
    
    # Remove extra whitespaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# Register the text processing function as a UDF
process_text_udf = udf(process_text, StringType())

# Apply the text processing UDF to the "Tweet" column
df = twitterDF.withColumn("clean_tweet", process_text_udf(twitterDF["Tweet"]))

In [8]:
# Get the current column names
current_columns = df.columns

# Create a new DataFrame with lowercase column names
df = df.toDF(*[col.lower() for col in current_columns])

In [9]:
df.show()

+----------+-----------+--------------------+--------------------+
|      date|       time|               tweet|         clean_tweet|
+----------+-----------+--------------------+--------------------+
|01/27/2017| 6:46:22 PM| I promise that o...|i promise that ou...|
|01/27/2017| 5:00:47 PM| Congratulations ...|congratulations s...|
|01/27/2017| 3:20:15 PM| Statement on Int...|statement on inte...|
|01/27/2017|11:30:29 AM| .@VP Mike Pence ...|mike pence will b...|
|01/27/2017|11:27:02 AM| The #MarchForLif...|the is so importa...|
|01/27/2017| 8:19:10 AM| Mexico has taken...|mexico has taken ...|
|01/27/2017| 8:12:52 AM| Look forward to ...|look forward to s...|
|01/26/2017| 6:53:37 PM| Miami-Dade Mayor...|miamidade mayor d...|
|01/26/2017| 6:45:28 PM| Will be intervie...|will be interview...|
|01/26/2017| 2:21:17 PM| Spoke at the Con...|spoke at the cong...|
|01/26/2017| 8:55:03 AM| of jobs and comp...|of jobs and compa...|
|01/26/2017| 8:51:46 AM| The U.S. has a 6...|the us has a 60 b

In [10]:
# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017')  # Replace 'localhost' with your MongoDB server address

# Access the database and collection
db = client['sentiment']  # Replace 'sentiment' with your database name
collection = db['sentiment_collection']  # Replace 'sentiment_collection' with your collection name

# Convert DataFrame to RDD and collect all rows
rows = df.rdd.collect() #!!!!here df has to be new_df if everything goes back!!!!

# Insert the dataset into the collection
for row in rows:
    collection.insert_one(row.asDict())

# Close the MongoDB connection
#client.close()


In [11]:
# Retrieve data from MongoDB collection, excluding the "_id" field
data = list(collection.find({}, {"_id": 0}))

# Create a DataFrame from the retrieved data
df = spark.createDataFrame(data)

# Apply sentiment analysis using NLTK
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(tweet):
    sentiment_score = analyzer.polarity_scores(tweet)['compound']
    
    if sentiment_score > 0:
        sentiment_label = "Positive"
    elif sentiment_score < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"
    
    return sentiment_score, sentiment_label

# Define the schema for the sentiment analysis result
sentiment_schema = StructType([
    StructField("sentiment_score", FloatType(), nullable=False),
    StructField("sentiment_label", StringType(), nullable=False)
])

sentiment_udf = udf(analyze_sentiment, sentiment_schema)

# Apply sentiment analysis and create new columns for sentiment score and label
df = df.withColumn("sentiment", sentiment_udf(df["clean_tweet"]))
df = df.withColumn("sentiment_score", df["sentiment"]["sentiment_score"])
df = df.withColumn("sentiment_label", df["sentiment"]["sentiment_label"])

# Select the desired columns
selected_columns = ["date", "time", "clean_tweet", "sentiment_score", "sentiment_label"]
df = df.select(*selected_columns)

# Show the DataFrame with sentiment analysis results
df.show()

# Close the MongoDB connection
#client.close()


+----------+-----------+--------------------+---------------+---------------+
|      date|       time|         clean_tweet|sentiment_score|sentiment_label|
+----------+-----------+--------------------+---------------+---------------+
|01/27/2017| 6:46:22 PM|i promise that ou...|         0.3182|       Positive|
|01/27/2017| 5:00:47 PM|congratulations s...|         0.5994|       Positive|
|01/27/2017| 3:20:15 PM|statement on inte...|            0.0|        Neutral|
|01/27/2017|11:30:29 AM|mike pence will b...|         0.4019|       Positive|
|01/27/2017|11:27:02 AM|the is so importa...|         0.6207|       Positive|
|01/27/2017| 8:19:10 AM|mexico has taken ...|         0.0552|       Positive|
|01/27/2017| 8:12:52 AM|look forward to s...|        -0.1779|       Negative|
|01/26/2017| 6:53:37 PM|miamidade mayor d...|         0.5106|       Positive|
|01/26/2017| 6:45:28 PM|will be interview...|         0.4939|       Positive|
|01/26/2017| 2:21:17 PM|spoke at the cong...|         0.2023|   

In [12]:
# Group by sentiments and count the number of tweets in each sentiment category
sentiment_counts = df.groupBy("sentiment_label").count().withColumnRenamed("count", "tweet_count")

# Show the resulting DataFrame
sentiment_counts.show()

+---------------+-----------+
|sentiment_label|tweet_count|
+---------------+-----------+
|       Positive|     125132|
|        Neutral|      36785|
|       Negative|      50778|
+---------------+-----------+



In [13]:
from pyspark.sql.functions import col, year, month, weekofyear, dayofmonth, hour, to_date, to_timestamp
#from pyspark.sql.functions import to_date, to_timestamp

#Convert date column to DateType
df = df.withColumn("date", to_date(df["date"], "MM/dd/yyyy"))

#Convert time column to TimestampType
df = df.withColumn("time", to_timestamp(df["time"], "h:mm:ss a"))

#Extract date components
df = df.withColumn("year",year(col("Date")))
df = df.withColumn("month",month(col("Date")))
df = df.withColumn("week",weekofyear(col("Date")))
df = df.withColumn("day",dayofmonth(col("Date")))

#Extract time components
df = df.withColumn("hour", hour(col("Time")))

df.show()

+----------+-------------------+--------------------+---------------+---------------+----+-----+----+---+----+
|      date|               time|         clean_tweet|sentiment_score|sentiment_label|year|month|week|day|hour|
+----------+-------------------+--------------------+---------------+---------------+----+-----+----+---+----+
|2017-01-27|1970-01-01 18:46:22|i promise that ou...|         0.3182|       Positive|2017|    1|   4| 27|  18|
|2017-01-27|1970-01-01 17:00:47|congratulations s...|         0.5994|       Positive|2017|    1|   4| 27|  17|
|2017-01-27|1970-01-01 15:20:15|statement on inte...|            0.0|        Neutral|2017|    1|   4| 27|  15|
|2017-01-27|1970-01-01 11:30:29|mike pence will b...|         0.4019|       Positive|2017|    1|   4| 27|  11|
|2017-01-27|1970-01-01 11:27:02|the is so importa...|         0.6207|       Positive|2017|    1|   4| 27|  11|
|2017-01-27|1970-01-01 08:19:10|mexico has taken ...|         0.0552|       Positive|2017|    1|   4| 27|   8|
|

In [14]:
sentiment_distribution_year = df.groupBy("year","sentiment_label").count().orderBy("year","sentiment_label")
sentiment_distribution_year.show()

+----+---------------+-----+
|year|sentiment_label|count|
+----+---------------+-----+
|2009|       Negative|   35|
|2009|        Neutral|  112|
|2009|       Positive|  245|
|2010|       Negative|   70|
|2010|        Neutral|  210|
|2010|       Positive|  714|
|2011|       Negative| 2037|
|2011|        Neutral| 1393|
|2011|       Positive| 1988|
|2012|       Negative| 8162|
|2012|        Neutral| 4186|
|2012|       Positive|12369|
|2013|       Negative|13384|
|2013|        Neutral| 7973|
|2013|       Positive|35651|
|2014|       Negative| 7903|
|2014|        Neutral| 6818|
|2014|       Positive|25823|
|2015|       Negative|10395|
|2015|        Neutral|11137|
+----+---------------+-----+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import avg, split, size

# Calculate average tweets per week
weekly_tweets = df.groupBy("year", "week").count()
average_tweets_per_week = weekly_tweets.groupBy("year").agg(avg("count").alias("average_tweets_per_week"))
average_tweets_per_week = average_tweets_per_week.orderBy("year")
average_tweets_per_week.show()

# Calculate average length per tweet (by word)
df_with_word_count = df.withColumn("word_count", size(split("clean_tweet", " ")))
average_length_per_tweet = df_with_word_count.agg(avg("word_count").alias("average_length_per_tweet"))
average_length_per_tweet.show()


+----+-----------------------+
|year|average_tweets_per_week|
+----+-----------------------+
|2009|                   14.0|
|2010|     20.285714285714285|
|2011|                 108.36|
|2012|      475.3269230769231|
|2013|     1096.3076923076924|
|2014|      779.6923076923077|
|2015|      994.5283018867924|
|2016|      558.0188679245283|
|2017|                  267.4|
+----+-----------------------+



+------------------------+
|average_length_per_tweet|
+------------------------+
|       14.43422741484285|
+------------------------+



In [16]:
import cassandra

# Check if cassandra-driver is installed
if 'cassandra' in dir(cassandra):
    print("Cassandra driver is installed.")
else:
    print("Cassandra driver is not installed.")


Cassandra driver is not installed.


In [17]:
import cassandra

In [18]:
spark.conf.set("spark.cassandra.connection.host", "127.0.0.1")  # Replace with your Cassandra host
spark.conf.set("spark.cassandra.connection.port", "9042")  # Replace with your Cassandra port

In [22]:
df.write \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="date_extracted_db", keyspace="my_keyspace") \
    .mode("Append") \
    .save()